<a href="https://colab.research.google.com/github/prasanth5reddy/D2L/blob/master/Modern%20Convolutional%20Networks/networks_using_blocks_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mounting Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import sys
w_dir = '/content/drive/My Drive/Colab/D2L.AI/'
sys.path.append(w_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Installing Libraries

In [2]:
!pip uninstall mxnet && pip install mxnet-cu100

Importing Libraries

In [0]:
from mxnet import gluon, init, nd
from mxnet.gluon import nn
import d2l

VGG Blocks

In [0]:
def vgg_block(num_convs, num_channels):
  blk = nn.Sequential()
  for _ in range(num_convs):
    blk.add(nn.Conv2D(num_channels, kernel_size=3, padding=1, activation='relu'))
  blk.add(nn.MaxPool2D(pool_size=2, strides=2))
  return blk

VGG Network

In [0]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

In [0]:
def vgg(conv_arch):
  net = nn.Sequential()
  for num_convs, num_channels in conv_arch:
    net.add(vgg_block(num_convs, num_channels))
  
  net.add(nn.Dense(4096, activation='relu'), nn.Dropout(0.5),
          nn.Dense(4096, activation='relu'), nn.Dropout(0.5),
          nn.Dense(10))
  return net

In [7]:
net = vgg(conv_arch)
net.initialize()
X = nd.random.uniform(shape=(1, 1, 224, 224))
for blk in net:
  X = blk(X)
  print(blk.name, 'output shape:\t', X.shape)

sequential1 output shape:	 (1, 64, 112, 112)
sequential2 output shape:	 (1, 128, 56, 56)
sequential3 output shape:	 (1, 256, 28, 28)
sequential4 output shape:	 (1, 512, 14, 14)
sequential5 output shape:	 (1, 512, 7, 7)
dense0 output shape:	 (1, 4096)
dropout0 output shape:	 (1, 4096)
dense1 output shape:	 (1, 4096)
dropout1 output shape:	 (1, 4096)
dense2 output shape:	 (1, 10)


Model Training

In [8]:
ratio = 4
small_conv_arch = [(pair[0], pair[1] // ratio) for pair in conv_arch]
small_conv_arch

[(1, 16), (1, 32), (2, 64), (2, 128), (2, 128)]

In [9]:
net = vgg(small_conv_arch)
lr, num_epochs, batch_size, ctx = 0.05, 5, 128, d2l.try_gpu()
net.initialize(ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)

training on gpu(0)
epoch 1, loss 0.9689, train acc 0.650, test acc 0.842, time 105.2 sec
epoch 2, loss 0.4105, train acc 0.849, test acc 0.884, time 101.3 sec
epoch 3, loss 0.3364, train acc 0.876, test acc 0.898, time 101.3 sec
epoch 4, loss 0.2929, train acc 0.894, test acc 0.908, time 101.4 sec
epoch 5, loss 0.2633, train acc 0.904, test acc 0.913, time 101.3 sec
